In [893]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import stats

In [894]:
leso = pd.read_csv('../data/1033-program-foia-may-2014.csv')

In [895]:
#LESO Dataframe EDA
leso.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243492 entries, 0 to 243491
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   State             243492 non-null  object 
 1   County            243422 non-null  object 
 2   NSN               243309 non-null  object 
 3   Item Name         239658 non-null  object 
 4   Quantity          243492 non-null  int64  
 5   UI                243491 non-null  object 
 6   Acquisition Cost  243492 non-null  float64
 7   Ship Date         243492 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 14.9+ MB


In [896]:
#Clean Data

#Capitalize case County names
leso['County'] = leso['County'].str.upper()

#Change Ship Date to Datetime
leso['Ship Date'] = pd.to_datetime(leso['Ship Date'], format='mixed')

print(leso.head())
print(leso.info())


  State     County               NSN              Item Name  Quantity    UI  \
0    AK  ANCHORAGE  1005-00-073-9421  RIFLE,5.56 MILLIMETER         1  Each   
1    AK  ANCHORAGE  1005-00-073-9421  RIFLE,5.56 MILLIMETER         1  Each   
2    AK  ANCHORAGE  1005-00-073-9421  RIFLE,5.56 MILLIMETER         1  Each   
3    AK  ANCHORAGE  1005-00-073-9421  RIFLE,5.56 MILLIMETER         1  Each   
4    AK  ANCHORAGE  1005-00-073-9421  RIFLE,5.56 MILLIMETER         1  Each   

   Acquisition Cost  Ship Date  
0             499.0 2012-08-30  
1             499.0 2012-08-30  
2             499.0 2012-08-30  
3             499.0 2012-08-30  
4             499.0 2012-08-30  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243492 entries, 0 to 243491
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   State             243492 non-null  object        
 1   County            243422 non-null  object      

In [897]:
leso = leso.dropna(subset=['Item Name'])

In [898]:
# #Duplicate Rows with Quantities greater than 1

# #leso['Quantity'].unique()

# def duplicate_rows(row):
#     if row['Quantity'] > 1:
#         return pd.concat([row] * row['Quantity'], ignore_index=True)
#     else:
#         return row
    
# foia = leso.apply(duplicate_rows, axis=1)


In [899]:
#Create DataFrame for Total sums of equipment and the cost
foia = leso.groupby(['State', 'County']).sum(numeric_only=True)

foia = foia.rename(columns = {'Quantity':'Total Equip Quantity' , 'Acquisition Cost': 'Total Cost'})

In [900]:
foia.head()

Total Equip Quantity  Total Cost
State County                                                
AK    ANCHORAGE                             1063   636550.09
      BETHEL                                   2        0.00
      FAIRBANKS NORTH STAR                    18    26624.67
      JUNEAU                                  14     5542.00
      KETCHIKAN GATEWAY                        4     1996.00

In [901]:
totals = foia.reset_index()
totals.head()

,State,County,Total Equip Quantity,Total Cost
0,AK,ANCHORAGE,1063,636550.09
1,AK,BETHEL,2,0.00
2,AK,FAIRBANKS NORTH STAR,18,26624.67
3,AK,JUNEAU,14,5542.00
4,AK,KETCHIKAN GATEWAY,4,1996.00


In [902]:
test_df = leso[leso['Item Name'].str.contains('GRENADE')]
test_df['Item Name'].value_counts()

Item Name
LAUNCHER,GRENADE                                        205
POUCH,FLASH BANG GRENADE                                 30
POUCH,GRENADE,HAND                                       19
POUCH,HAND GRENADE                                       18
POUCH,HAND GRENADE                                       16
POUCH,FRAG GRENADE,SINGLE                                11
POUCH,GRENADE,HAND                                        6
POUCH,GRENADE                                             5
HAND,GRENADE POUCH                                        5
POUCH,FRAG GRENADE,                                       4
BOX,GRENADE                                               4
BARREL,GRENADE LAUNCHER                                   3
POUCH,SF,GRENADE,KH                                       3
POUCH,GRENADE                                             3
POUCH,SMOKE GRENADE                                       2
CPCVX HOLSTER GRENADE POUCH                               2
GRENADE POUCH,MAINTENANCE OF O

In [903]:
#Filter for militarized equipment

#Equipment Bools
weapon_bool = (leso['Item Name'] == 'RIFLE,7.62 MILLIMETER') | (leso['Item Name'] == 'RIFLE,5.56 MILLIMETER') | (leso['Item Name'] == 'SHOTGUN,12 GAGE,RIOT TYPE') | (leso['Item Name'] == 'LAUNCHER,GRENADE')
rifle_df = leso[weapon_bool]
rifle_df['Item Name'].value_counts()

vehicle_bool = (leso['Item Name'] == 'MINE RESISTANT VEHICLE') | (leso['Item Name'] == 'LIGHT ARMORED VEHICLE') | (leso['Item Name'] == 'ONLY COMPLETE COMBAT/ASSAULT/TACTICAL WHEELED VEHICLES') |(leso['Item Name'] == 'UTILITY VEHICLE,4WD') | (leso['Item Name'] == 'HELICOPTER,OBSERVATION') | (leso['Item Name'] == 'HELICOPTER,UTILITY') | (leso['Item Name'] == 'TRUCK,ARMORED')
vic_df = leso[vehicle_bool]
vic_df['Item Name'].value_counts()

helmet_bool = (leso['Item Name'] == 'HELMET,ADVANCED COMBAT') | (leso['Item Name'] == 'ARMOR, HELMET') | (leso['Item Name'] == "HELMET,GROUND TROOPS'") | (leso['Item Name'] == 'HELMET,GROUND TROOPS') | (leso['Item Name'] == 'HELMET,GROUND TROOPS-PARACHUTISTS')
helmet_df = leso[helmet_bool]

night_vis_bool = (leso['Item Name'] == 'NIGHT VISION GOGGLE ') | (leso['Item Name'] == 'VIEWER,NIGHT VISION') | (leso['Item Name'] == 'IMAGE INTENSIFIER,NIGHT VISION') | (leso['Item Name'] == 'NIGHT VISION EQUIP, EMIT, REFLECTED RAD') | (leso['Item Name'] == 'NIGHT VISION SIGHT') | (leso['Item Name'] == 'NIGHT VISION SIGHT INDIVIDUAL SERVED WEAPONS') | (leso['Item Name'] == 'NIGHT VISION DEVICE') | (leso['Item Name'] == 'GOGGLES,NIGHT VISION') | (leso['Item Name'] == 'BINOCULAR,NIGHT VISION') | (leso['Item Name'] == 'SIGHT,NIGHT VISION') | (leso['Item Name'] == 'NIGHT VISION SIGHT,') | (leso['Item Name'] == 'VIEWER KIT,NIGHT VISION')
night_vis_df = leso[night_vis_bool]

#specialty equipment
spec_equip_bool = (leso['Item Name'] == 'SIGHT,NIGHT VISION SNIPERSCOPE') | (leso['Item Name'] == 'TELESCOPE,STRAIGHT') | (leso['Item Name'] == 'TELESCOPE,NON-INVERTING INFRARED') | (leso['Item Name'] == 'SCOPE,NIGHT VISION')
spec_df = leso[spec_equip_bool]

#armor
armor_bool = (leso['Item Name'] == 'ARMOR, PERSONAL') | (leso['Item Name'] == 'ARMOR,SUPPLEMENTAL,SMALL ARMS-FRAGMENTATION PROTECTIVE') | (leso['Item Name'] == 'ARMOR PLATE') | (leso['Item Name'] == 'ARMOR SET,SUPPLEMENTAL,SMALL ARMS-FRAGMENTATION PROTECTIVE') | (leso['Item Name'] == 'BODY ARMOR,FRAGMENTATION PROTECTIVE') | (leso['Item Name'] == 'ARMOR, PLATE') 
armor_df = leso[armor_bool]
# ARMOR, PERSONAL                                                            47
# ARMOR,SUPPLEMENTAL,SMALL ARMS-FRAGMENTATION PROTECTIVE                     25
# ARMOR PLATE
# ARMOR SET,SUPPLEMENTAL,SMALL ARMS-FRAGMENTATION PROTECTIVE                 10
# BODY ARMOR,FRAGMENTATION PROTECTIVE 
# INSERT,SMALL ARMS PROTECTIVE BODY ARMOR
# ARMOR, PLATE

frames = [vic_df, rifle_df, helmet_df, night_vis_df, spec_df, armor_df]

mil_equip = pd.concat(frames)
mil_equip.info()

<class 'pandas.core.frame.DataFrame'>
Index: 86925 entries, 146 to 241137
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   State             86925 non-null  object        
 1   County            86925 non-null  object        
 2   NSN               86882 non-null  object        
 3   Item Name         86925 non-null  object        
 4   Quantity          86925 non-null  int64         
 5   UI                86925 non-null  object        
 6   Acquisition Cost  86925 non-null  float64       
 7   Ship Date         86925 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 6.0+ MB


In [904]:
#Create DataFrame for Total sums of Militarized equipment and the cost
militarized = mil_equip.groupby(['State', 'County']).sum(numeric_only=True)

militarized = militarized.rename(columns = {'Quantity':'Mil Equip Quantity' , 'Acquisition Cost': 'Total Mil Cost'})

In [905]:
militarized.sample(20)

,,Mil Equip Quantity,Total Mil Cost
State,County,,
WI,FOND DU LAC,59,16771.0
GA,EARLY,2,4700.0
CO,LAS ANIMAS,4,534.0
AL,MORGAN,20,721920.0
GA,DECATUR,26,4640.0
MN,BIG STONE,7,1616.0
TN,LAWRENCE,20,2760.0
KS,MIAMI,37,11388.0
NY,CAYUGA,4,552.0


In [906]:
militarized = militarized.reset_index()
militarized.head()

,State,County,Mil Equip Quantity,Total Mil Cost
0,AK,ANCHORAGE,130,521602.23
1,AK,FAIRBANKS NORTH STAR,15,7485.00
2,AK,JUNEAU,14,5542.00
3,AK,KETCHIKAN GATEWAY,4,1996.00
4,AK,KODIAK ISLAND,5,690.00


In [907]:
militarized.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2304 entries, 0 to 2303
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   State               2304 non-null   object 
 1   County              2304 non-null   object 
 2   Mil Equip Quantity  2304 non-null   int64  
 3   Total Mil Cost      2304 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 72.1+ KB


In [908]:
#Create a merged DataFrame that accounts for both total amounts of equipment and militarized equipment
merged_equipment = pd.merge(totals, militarized, on=['State', 'County'], how='outer')
merged_equipment.fillna(0, inplace=True)

In [909]:
merged_equipment['County'] = merged_equipment['County'].str.replace("SAINT", "ST")
merged_equipment['County'] = merged_equipment['County'].str.replace('-', ' ')
merged_equipment['County'] = merged_equipment['County'].str.replace('DEKALB', 'DE KALB')
merged_equipment['County'] = merged_equipment['County'].str.replace('DEWITT', 'DE WITT')
merged_equipment['County'] = merged_equipment['County'].str.replace('DUPAGE', 'DU PAGE')
merged_equipment['County'] = merged_equipment['County'].str.replace('DESOTO', 'DE SOTO')


In [910]:
merged_equipment.sample(10)

,State,County,Total Equip Quantity,Total Cost,Mil Equip Quantity,Total Mil Cost
771,KS,COMANCHE,1,138.00,1.0,138.00
914,KY,TODD,11,52754.00,8.0,1754.00
1343,MT,JUDITH BASIN,1,499.00,1.0,499.00
2214,TX,MILLS,3,1136.00,3.0,1136.00
1318,MS,UNION,1,499.00,1.0,499.00
813,KS,PRATT,107,35051.06,24.0,5478.00
374,GA,DOOLY,42,129169.29,7.0,966.00
10,AL,BIBB,30,39047.72,24.0,17579.72
103,AR,LONOKE,513,1090941.43,39.0,13111.00
966,LA,VERMILION,29,7902.10,19.0,7315.00


In [911]:
police = pd.read_csv('../data/police_fatalities 2.csv')

In [912]:
police.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28335 entries, 0 to 28334
Data columns (total 29 columns):
 #   Column                                                          Non-Null Count  Dtype  
---  ------                                                          --------------  -----  
 0   Unique ID                                                       28334 non-null  object 
 1   Subject's name                                                  28335 non-null  object 
 2   Subject's age                                                   27354 non-null  object 
 3   Subject's gender                                                28237 non-null  object 
 4   Subject's race                                                  28334 non-null  object 
 5   Subject's race with imputations                                 27905 non-null  object 
 6   Imputation probability                                          27897 non-null  object 
 7   URL of image of deceased                         

In [913]:
police['Cause of death'].unique()

array(['Vehicle', 'Gunshot', 'Beaten/Bludgeoned with instrument',
       'Stabbed', 'Asphyxiated/Restrained', 'Drowned', 'Drug overdose',
       'Fell from a height', 'Undetermined',
       'Chemical agent/Pepper spray', 'Medical emergency', 'Other',
       'Burned/Smoke inhalation', 'Tasered', 'Unknown', 'Pursuit', nan],
      dtype=object)

In [914]:
police_df = police.drop(columns = ["Full Address", "Subject's name", "Unique identifier (redundant)", "Dispositions/Exclusions INTERNAL USE, NOT FOR ANALYSIS", \
                       "Link to news article or photo of official document", "Symptoms of mental illness? INTERNAL USE, NOT FOR ANALYSIS", \
                        "Video", "Unique ID formula", "Date&Description", "URL of image of deceased", "URL of image of deceased", "Subject's age", \
                            "Subject's gender", "Subject's race", "Subject's race with imputations", "Imputation probability", "A brief description of the circumstances surrounding the death", \
                                'Location of injury (address)'])

In [915]:
police_df.rename(columns = {"Location of death (county)": "County", "Location of death (city)": "City", \
                            "Location of death (state)": "State", "Location of death (zip code)": "Zip Code"}, inplace=True)

In [916]:
police_df.head()

,Unique ID,Date of injury resulting in death (month/day/year),City,State,Zip Code,County,Latitude,Longitude,Agency responsible for death,Cause of death,Intentional Use of Force (Developing),Date (Year)
0,25746,01/01/2000,Willits,CA,95490.0,Mendocino,39.470883,-123.361751,Mendocino County Sheriff's Office,Vehicle,Vehicle/Pursuit,2000.0
1,25747,01/01/2000,Detroit,MI,48203.0,Wayne,42.404526,-83.092274,NaN,Vehicle,Vehicle/Pursuit,2000.0
2,25748,01/01/2000,Detroit,MI,48203.0,Wayne,42.404526,-83.092274,NaN,Vehicle,Vehicle/Pursuit,2000.0
3,25749,01/01/2000,Carlsbad,NM,88220.0,Eddy,32.450080,-104.237643,Eddy County Sheriff's Office,Vehicle,Vehicle/Pursuit,2000.0
4,2,01/02/2000,Ellenwood,GA,30294.0,DeKalb,33.645164,-84.229413,DeKalb County Sheriff's Office,Gunshot,"Intentional Use of Force, Deadly",2000.0


In [917]:
police_df['County'] = police_df['County'].str.upper()
police_df['County'] = police_df['County'].str.replace("'", "")
police_df['County'] = police_df['County'].str.replace("ST.","ST")
police_df['County'] = police_df['County'].str.replace("STE.","STE")
police_df['County'] = police_df['County'].str.replace("SAINT","ST")
police_df['County'] = police_df['County'].str.replace('-', ' ')
#Fix Individual Counties
police_df['County'] = police_df['County'].str.replace('DEKALB', 'DE KALB')
police_df['County'] = police_df['County'].str.replace('PARK HILL', 'DENVER')
police_df['County'] = police_df['County'].str.replace('DESOTO', 'DE SOTO')
police_df['County'] = police_df['County'].str.replace('OSKALOOSA', 'OKALOOSA')
police_df['County'] = police_df['County'].str.replace('JOHNSON ', 'JOHNSON')
police_df['County'] = police_df['County'].str.replace('CHICAGO', 'COOK')
police_df['County'] = police_df['County'].str.replace('LASALLE', 'LA SALLE')
police_df['County'] = police_df['County'].str.replace('DUPAGE', 'DU PAGE')
police_df['County'] = police_df['County'].str.replace('VERMILLION', 'VERMILION')
police_df['County'] = police_df['County'].str.replace('MCCLEAN', 'MCLEAN')
police_df['County'] = police_df['County'].str.replace('LAPORTE', 'LA PORTE')
police_df['County'] = police_df['County'].str.replace('INDIANAPOLIS', 'MARION')
police_df['County'] = police_df['County'].str.replace('WASHENTAW', 'WASHTENAW')

police_df['County'] = police_df['County'].str.replace(' CENSUS AREA', '')
police_df['County'] = police_df['County'].str.replace(' COUNTY', '')
# police_df['County'] = police_df['County'].str.replace('WESAINT', 'WEST')
# police_df['County'] = police_df['County'].str.replace('EASAINT


In [918]:
fatal_counties = police_df.groupby(['County', 'State']).size()
#fatal_counties.head()
fatal_counties = pd.DataFrame(fatal_counties)


In [919]:
fatalities = fatal_counties.reset_index()
police_victims = fatalities.rename(columns = {0: 'Fatalities'})
police_victims.head(10)

,County,State,Fatalities
0,ACADIA,LA,7
1,ACCOMACK,VA,2
2,ADA,ID,31
3,ADAIR,OK,6
4,ADAMS,CO,78
5,ADAMS,ID,2
6,ADAMS,IL,5
7,ADAMS,MS,6
8,ADAMS,NE,2
9,ADAMS,OH,1


In [920]:
police_victims.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2372 entries, 0 to 2371
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   County      2372 non-null   object
 1   State       2372 non-null   object
 2   Fatalities  2372 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 55.7+ KB


In [921]:
#Create a merged DataFrame that accounts for both total amounts of equipment and militarized equipment
analysis = pd.merge(merged_equipment, police_victims, on=['State', 'County'], how='outer')
analysis.fillna(0, inplace=True)

In [922]:
pd.options.display.max_rows = 250

In [929]:
state = analysis[analysis['State'] == 'NY']
state

,State,County,Total Equip Quantity,Total Cost,Mil Equip Quantity,Total Mil Cost,Fatalities
1615,NY,ALBANY,188.0,4266391.82,149.0,2148989.00,12.0
1616,NY,ALLEGANY,16.0,115479.55,10.0,68117.00,0.0
1617,NY,BROOME,30.0,804583.42,16.0,462070.00,5.0
1618,NY,CAYUGA,20.0,72520.35,4.0,552.00,1.0
1619,NY,CHEMUNG,27.0,49784.97,0.0,0.00,3.0
1620,NY,CHENANGO,6.0,51274.00,4.0,1274.00,3.0
1621,NY,CLINTON,21.0,1380714.09,1.0,658000.00,1.0
1622,NY,DELAWARE,4.0,96254.00,2.0,276.00,3.0
1623,NY,DUTCHESS,4.0,552.00,4.0,552.00,9.0
1624,NY,ERIE,13.0,973332.12,7.0,658828.00,20.0
